# Gather data

### Import necessary libraries

In [1]:
import tweepy
import pandas as pd
import requests
import io
import json
import os
import glob
import numpy as np
import re
from datetime import datetime as dt

### Initialize twitter API access

In [2]:

consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth ,wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

### Import twitter archive of user WeRateDogs

In [3]:
df = pd.read_csv("twitter-archive-enhanced.csv")

### Gather additional tweet data and save it in a folder

In [4]:
# Check which staus info is already collected
folder_name = "tweet_data"
ids_collected = list(map(lambda x: int(x[11:-4]), glob.glob(folder_name+"/*.txt")))

In [5]:
def get_tweet_status():
    '''
    Takes the status ids of the twitter archive csv and queries the twitter api for the complete information.
    Saves a text file with the information for each twitter status. Also saves a list of statuses that were 
    not able to be retrieved from the API
    
    Args:
        None
    Output:
        None
    '''
    # Create a folder to store all the tweet data inside
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Error list for possibly deleted tweets
    error_list = []
    # Variable to measure progress
    i = 1
    # Get the status for each tweet id in the tweet_id column of the twitter archive
    
    for tweet_id in list(df.tweet_id):
        
        # Print progress status for every 20th call
        if i%20 ==0:
            print(str(i/len(df)*100)+"% of API download done")
        
        # If the id is already collected it should be skipped to save time
        if tweet_id not in ids_collected:
            try:
                tweet = api.get_status(tweet_id)
                data = tweet._json

                #Save the json data as a txt file with the corresponding tweet id (Saving as .txt to fulfill project rubric)
                with open(folder_name+"/"+str(tweet_id)+'.txt', 'w', encoding='utf-8') as f:
                    json.dump(data, f, ensure_ascii=False, indent=4)
            except Exception as e:
                error_list.append(tweet_id)
                print("The status of the tweet with id {} is not possible to access".format(tweet_id))
            # Store the tweet json data in a variable
            
                
        i+=1
        
    # Save the error list for later usage    
    pd.DataFrame(error_list).to_csv("na_ids.csv", index = False)
    

In [6]:
# Only used once to get the data
#get_tweet_status()

### Import tweet status data pulled from the API as text and write it in a dataframe
Comment: I saved the data as txt to fulfill the rubric of this project to work with three different kind of data.
It would have been a lot easier to query the json structure than this regex stuff which is quite an unrobust and dirty solution I think.

In [7]:
def text_to_df():  
    '''
    Takes the text files from the subfolder and saves the tweet id,
    follower_count, favourites of user count, retweet count and the 
    favourites of the post count in a dataframe.
    
    Args:
        None
    Output:
        Saves the data as a dataframe, returns no output.
    '''
    # Initialize list of dictionaries
    df_list = []

    for status in glob.glob(folder_name+"/*.txt"):
        with open(status, encoding = "utf-8") as file:
            # Read only the couple of first lines for the id, afterwards the files differ from each other
            lines = file.readlines()[2:4]
            file.seek(0)
            text = file.read()
            text_id = lines[1].strip()[11:-2]

            # Be sure that ids are consistent
            if text_id != status[11:-4]:
                raise Exception("Id saved in file does not match id from filename")

            # Find the needed data with regex, since line by line reading not possible
            follower_count = re.findall(r"\"followers_count\": (\d+),", text)[0]
            favourites_of_user_count = re.findall(r"\"favourites_count\": (\d+),", text)[0]
            favourite_of_post_count = re.findall(r"\"favorite_count\": (\d+),", text)[0]
            retweet_count = re.findall(r"\"retweet_count\": (\d+),", text)[0]

            # Append the list entry of dictionaries to create a dataframe
            df_list.append({"tweet_id": text_id,
                           "follower_count": follower_count,
                           "favourites_of_user_count": favourites_of_user_count,
                           "retweet_count": retweet_count,
                           "favourite_of_post_count": favourite_of_post_count})

            file.close()


    df_additional_data = pd.DataFrame(df_list, columns = ["tweet_id", "follower_count", "favourites_of_user_count", "retweet_count", "favourite_of_post_count"])
    df_additional_data.to_csv("api_twitter_status.csv")

In [8]:
# Only needed to run once to create the df
#text_to_df()

### Download the dog predictions file

In [9]:
r = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv').content

dog_predictions = pd.read_csv(io.StringIO(r.decode('utf-8')), sep = "\t")

dog_predictions.to_csv("dog_predictions.csv")

# Asses
Now that all needed data is gathered the three data frames are assessed.
The dataframes are:
 - twitter-archives-enhanced.csv
 - api_twitter_status.csv
 - dog_predictions.csv
 
 *All found issues are recorded at the bottom of this chapter*


In [10]:
twitter_df = pd.read_csv("twitter-archive-enhanced.csv")
api_df = pd.read_csv("api_twitter_status.csv", index_col = 0)
dogs_df = pd.read_csv("dog_predictions.csv", index_col = 0)
# List of statuses that were not available over the api
na_id = pd.read_csv("na_ids.csv")

In [11]:
twitter_df.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
96,873580283840344065,NaN,NaN,2017-06-10 16:39:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",We usually don't rate Deck-bound Saskatoon Bla...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/873580283...,13,10,None,None,None,None,None
1032,745314880350101504,NaN,NaN,2016-06-21 17:58:09 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Sugar. She excels underwater. 12/10 ph...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/745314880...,12,10,Sugar,None,None,None,None
1201,716447146686459905,NaN,NaN,2016-04-03 02:08:05 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This is Ozzy. He's acrobatic af. Legendary pup...,NaN,NaN,NaN,https://vine.co/v/eMmXVPn5eQK,13,10,Ozzy,None,None,pupper,None
425,821421320206483457,NaN,NaN,2017-01-17 18:17:58 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Sampson. He just gradua...,7.823059e+17,4.196984e+09,2016-10-01 19:47:08 +0000,https://twitter.com/dog_rates/status/782305867...,12,10,Sampson,doggo,None,None,None
1769,678380236862578688,NaN,NaN,2015-12-20 01:03:46 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Crumpet. He underestimated the snow. Q...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/678380236...,10,10,Crumpet,None,None,None,None


In [12]:
twitter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [13]:
type(twitter_df.timestamp[0])

str

In [14]:
twitter_df.in_reply_to_status_id.value_counts(dropna = False)

NaN             2278
6.671522e+17       2
6.737159e+17       1
6.753494e+17       1
6.754971e+17       1
                ... 
8.707262e+17       1
8.482121e+17       1
6.715449e+17       1
6.936422e+17       1
8.406983e+17       1
Name: in_reply_to_status_id, Length: 78, dtype: int64

In [15]:
twitter_df.source.value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

In [16]:
twitter_df.duplicated().sum()

0

In [17]:
api_df.sample(5)

,tweet_id,follower_count,favourites_of_user_count,retweet_count,favourite_of_post_count
8,714962719905021952,8709216,144674,4274,7382
1304,705898680587526145,8709217,144674,555,2372
744,668542336805281792,8709246,144674,207,442
1164,786729988674449408,8709207,144674,2490,8629
955,771136648247640064,8709208,144674,2988,9353


In [18]:
api_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2331 entries, 0 to 2330
Data columns (total 5 columns):
tweet_id                    2331 non-null int64
follower_count              2331 non-null int64
favourites_of_user_count    2331 non-null int64
retweet_count               2331 non-null int64
favourite_of_post_count     2331 non-null int64
dtypes: int64(5)
memory usage: 109.3 KB


In [19]:
api_df.duplicated().sum()

0

In [20]:
dogs_df.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
933,703356393781329922,https://pbs.twimg.com/media/CcLS6QKUcAAUuPa.jpg,1,Border_collie,0.894842,True,collie,0.097364,True,English_springer,0.003037,True
90,667538891197542400,https://pbs.twimg.com/media/CUOTFZOW4AABsfW.jpg,1,Yorkshire_terrier,0.618957,True,silky_terrier,0.300313,True,Australian_terrier,0.053412,True
594,679503373272485890,https://pbs.twimg.com/media/CW4UtmYWsAAEjqA.jpg,1,porcupine,0.999846,False,meerkat,0.000072,False,echidna,0.000044,False
932,703268521220972544,https://pbs.twimg.com/media/CcKC-5LW4AAK-nb.jpg,1,wool,0.525434,False,fur_coat,0.236391,False,kuvasz,0.038243,True
54,667044094246576128,https://pbs.twimg.com/media/CUHREBXXAAE6A9b.jpg,1,golden_retriever,0.765266,True,Labrador_retriever,0.206694,True,seat_belt,0.010667,False


In [21]:
dogs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 168.2+ KB


In [22]:
dogs_df.p1.value_counts()

golden_retriever      150
Labrador_retriever    100
Pembroke               89
Chihuahua              83
pug                    57
                     ... 
lawn_mower              1
polecat                 1
robin                   1
traffic_light           1
giant_panda             1
Name: p1, Length: 378, dtype: int64

In [23]:
dogs_df.duplicated().sum()

0

## Recording of issues
### Quality issues (dirty data, content issues)
 - Completeness: Is something missing?
 - Validity: Records are present but are faulty or in the wrong scheme
 - Accuracy: Records that are valid but have inaccurate measurements
 - Consistency: Technical correct but in different formats across tables or rows
#### twitter table
 - Retweets are included although they are not of interest
 - Replies to other statuses included although they are not of interest
 - Replies to users included although they are not of interest
 - Source column does not hold any valuable information
 - Timestamp is of type string
 - tweet_id is of type int
 - Remove columns without information
#### api table
 - tweet_id is of type int
 - Has missing tweets compared to the twitter table
#### dogs table
 - Naming of dog type is incoherent
 - tweet_id is of type int
 - Has missing tweets compared to the twitter table
### Tidyness iddues (structural issues)
 - Type of dog in the twitter table is streched over the last four columns although it is one variable
 - The twitter table and the api table are the same type of an observational unit (a twitter post)


# Cleaning the data

In [24]:
t_clean = twitter_df.copy()
a_clean = api_df.copy()
d_clean = dogs_df.copy()

### `twitter and api table` are the same type of an observational unit and api table has missing tweets
#### Define
Merge the two tables together on the tweet_id. Since the api table has less observations an inner merge on the ids available in both tables resolves two issues at the same time.

#### Code

In [25]:
t_clean.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


In [26]:
a_clean.head(1)

,tweet_id,follower_count,favourites_of_user_count,retweet_count,favourite_of_post_count
0,684588130326986752,8709217,144674,1419,4146


In [27]:
status_df = t_clean.merge(a_clean, how = "inner", on = "tweet_id")

#### Test

In [28]:
status_df.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,rating_denominator,name,doggo,floofer,pupper,puppo,follower_count,favourites_of_user_count,retweet_count,favourite_of_post_count
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,...,10,Phineas,None,None,None,None,8709203,144674,7776,36526


In [29]:
status_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2331 entries, 0 to 2330
Data columns (total 21 columns):
tweet_id                      2331 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2331 non-null object
source                        2331 non-null object
text                          2331 non-null object
retweeted_status_id           163 non-null float64
retweeted_status_user_id      163 non-null float64
retweeted_status_timestamp    163 non-null object
expanded_urls                 2272 non-null object
rating_numerator              2331 non-null int64
rating_denominator            2331 non-null int64
name                          2331 non-null object
doggo                         2331 non-null object
floofer                       2331 non-null object
pupper                        2331 non-null object
puppo                         2331 non-null object
follower_count                23

### `twitter table` (now status_df table) Type of dog in the twitter table is streched over the last four columns although it is one variable
#### Define
Make a new column with a variable for the type of dog and get rid of the four type of dog columns.

#### Code

In [30]:
status_df.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,rating_denominator,name,doggo,floofer,pupper,puppo,follower_count,favourites_of_user_count,retweet_count,favourite_of_post_count
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,...,10,Phineas,None,None,None,None,8709203,144674,7776,36526


In [31]:
# If no dogtype is present one of the dogtype gets an entry so it is easier to clean after melting
for i in range(len(status_df)):
    if status_df.iloc[i,13]==status_df.iloc[i,14]==status_df.iloc[i,15]==status_df.iloc[i,16]:
        status_df.loc[i, "doggo"] = "no dogtype"

In [32]:
status_df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,rating_denominator,name,doggo,floofer,pupper,puppo,follower_count,favourites_of_user_count,retweet_count,favourite_of_post_count
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,...,10,Phineas,no dogtype,None,None,None,8709203,144674,7776,36526
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,...,10,Tilly,no dogtype,None,None,None,8709202,144674,5762,31472
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,...,10,Archie,no dogtype,None,None,None,8709202,144674,3814,23715
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,...,10,Darla,no dogtype,None,None,None,8709203,144674,7948,39850
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,...,10,Franklin,no dogtype,None,None,None,8709202,144674,8574,38047


In [33]:
# Create a list for all the columns that should stay the same during melting
id_columns = list(status_df.columns[:13])
for item in list(status_df.columns[17:]):
    id_columns.append(item)

In [34]:
status_df = pd.melt(status_df, id_vars = id_columns, var_name = "dog_type", value_name='dog')

In [35]:
status_df = status_df.query("dog != 'None'")
len(status_df)

2345

In [36]:
# show all duplicates
len(status_df[status_df.tweet_id.duplicated(keep = False)].sort_values(by="tweet_id"))

28

In [37]:
# 14 statuses included two types of dogtype. Since it is ambigious which type is referencing the pictures all are removed
# from the dataset. This is ok since it is only a small number

In [38]:
# Get a list of the tweet ids with duplicates
duplicates = list(status_df[status_df.tweet_id.duplicated(keep = False)].tweet_id)

In [39]:
# Remove all rows whose tweet ids are in the duplicates list
status_df = status_df[~status_df['tweet_id'].isin(duplicates)]

In [40]:
for i in range(len(status_df)):
    if status_df.iloc[i,-1]=="no dogtype":
        status_df.iloc[i, -2] = "no dogtype"

In [41]:
status_df = status_df.drop(columns = "dog")

#### Test

In [42]:
status_df.dog_type.value_counts()

no dogtype    1956
pupper         243
doggo           80
puppo           29
floofer          9
Name: dog_type, dtype: int64

In [43]:
status_df.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,follower_count,favourites_of_user_count,retweet_count,favourite_of_post_count,dog_type
420,820314633777061888,NaN,NaN,2017-01-14 17:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",We are proud to support @LoveYourMelon on thei...,NaN,NaN,NaN,"https://www.loveyourmelon.com/pages/ourstory,h...",14,10,None,8709206,144674,576,3415,no dogtype
640,791672322847637504,NaN,NaN,2016-10-27 16:06:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When she says you're a good boy and you know y...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/791672322...,13,10,None,8709206,144674,3242,12114,no dogtype
684,786036967502913536,NaN,NaN,2016-10-12 02:53:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Scout. He really wants ...,7.798343e+17,4.196984e+09,2016-09-25 00:06:08 +0000,https://twitter.com/dog_rates/status/779834332...,11,10,Scout,8709208,144674,7253,19417,no dogtype
1057,738885046782832640,NaN,NaN,2016-06-04 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Charles. He's a Nova Scotian Towel Pou...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/738885046...,11,10,Charles,8709215,144674,1148,3800,no dogtype
1376,699446877801091073,NaN,NaN,2016-02-16 04:15:05 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He's undercover in all these p...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/699446877...,12,10,Archie,8709217,144674,2632,5994,no dogtype


### `twitter table` (now status table) 
 - Retweets are included although they are not of interest
 - Replies to other statuses included although they are not of interest
 - Replies to users included although they are not of interest
#### Define
Remove all retweets and replies of and to other statuses and users by identifying rows that hold values that are not NaN
#### Code

In [44]:
stat_df = status_df.copy()

In [45]:
len(stat_df)

2317

In [46]:
stat_df = stat_df[stat_df['in_reply_to_status_id'].isna()]

In [47]:
len(stat_df)

2240

In [48]:
stat_df = stat_df[stat_df['in_reply_to_user_id'].isna()]

len(stat_df)

2240

In [49]:
stat_df = stat_df[stat_df['retweeted_status_id'].isna()]

len(stat_df)

2079

In [50]:
stat_df = stat_df[stat_df['retweeted_status_user_id'].isna()]

len(stat_df)

2079

#### Test

In [51]:
stat_df.in_reply_to_status_id.value_counts()

Series([], Name: in_reply_to_status_id, dtype: int64)

In [52]:
stat_df.in_reply_to_user_id.value_counts()

Series([], Name: in_reply_to_user_id, dtype: int64)

In [53]:
stat_df.retweeted_status_id.value_counts()

Series([], Name: retweeted_status_id, dtype: int64)

In [54]:
stat_df.retweeted_status_user_id.value_counts()

Series([], Name: retweeted_status_user_id, dtype: int64)

#### Define 
Remove the columns that do not hold any information anymore. (The four columns tested above and the source column)

#### Code

In [55]:
stat_df.drop(columns = ["in_reply_to_status_id", "in_reply_to_user_id", "retweeted_status_id", "retweeted_status_user_id", "retweeted_status_timestamp", "source"], inplace = True)

In [56]:
stat_df.head(5)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,follower_count,favourites_of_user_count,retweet_count,favourite_of_post_count,dog_type
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,8709203,144674,7776,36526,no dogtype
1,892177421306343426,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,8709202,144674,5762,31472,no dogtype
2,891815181378084864,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,8709202,144674,3814,23715,no dogtype
3,891689557279858688,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,8709203,144674,7948,39850,no dogtype
4,891327558926688256,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,8709202,144674,8574,38047,no dogtype


### `all tables` tweet_id is of type int
### `stat table` timestamp is string
#### Define
Change all tweet_id columns from int to string.
Change the timestamp format from string to datetime

#### Code

In [57]:
stat_df.head(1)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,follower_count,favourites_of_user_count,retweet_count,favourite_of_post_count,dog_type
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,8709203,144674,7776,36526,no dogtype


In [58]:
stat_df["tweet_id"] = stat_df["tweet_id"].astype(str)
d_clean["tweet_id"] = d_clean["tweet_id"].astype(str)

In [59]:
stat_df["timestamp"] = pd.to_datetime(stat_df["timestamp"])

#### Test

In [60]:
d_clean.tweet_id[0]

'666020888022790149'

In [61]:
stat_df.tweet_id[0]

'892420643555336193'

In [62]:
stat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2079 entries, 0 to 8052
Data columns (total 13 columns):
tweet_id                    2079 non-null object
timestamp                   2079 non-null datetime64[ns, UTC]
source                      2079 non-null object
text                        2079 non-null object
expanded_urls               2076 non-null object
rating_numerator            2079 non-null int64
rating_denominator          2079 non-null int64
name                        2079 non-null object
follower_count              2079 non-null int64
favourites_of_user_count    2079 non-null int64
retweet_count               2079 non-null int64
favourite_of_post_count     2079 non-null int64
dog_type                    2079 non-null object
dtypes: datetime64[ns, UTC](1), int64(6), object(6)
memory usage: 307.4+ KB


# Storing the data

In [63]:
stat_df.to_csv("tweets_clean.csv", index = False)
d_clean.to_csv("dog_predictions_clean.csv", index = False)

# Analysis
 - Which dog breeds get the most likes by users (favourites and retweets)
 - Which dog breeds get the best ratings by the WeRateDogs
 - How does the account popularity develop over time

#### Which dog breeds get the most likes by users?

In [64]:
tweets = pd.read_csv("tweets_clean.csv")
dogs = pd.read_csv("dog_predictions_clean.csv")

In [65]:
dogs.head(1)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True


In [66]:
# Merge the two dataframes
df = tweets.merge(dogs, how="inner", on = "tweet_id")

In [67]:
df.head(1)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,follower_count,favourites_of_user_count,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,892420643555336193,2017-08-01 16:23:56+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,8709203,144674,...,1,orange,0.097049,False,bagel,0.085851,False,banana,0.07611,False
